In [1]:
import torchvision
from PIL import Image
import torchvision.transforms as T
import cv2
import matplotlib.pyplot as plt
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
!pip install torch-snippets==0.426

     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 57 kB 5.0 MB/s 
     |████████████████████████████████| 209 kB 42.4 MB/s 
     |████████████████████████████████| 51 kB 7.4 MB/s 


# Cargar video

In [14]:
# Detectando objetos en video
!gdown https://drive.google.com/uc?id=1xzNrYAqii3p-Y8XIiBd1ciSM4e1VRXIc

Downloading...
From: https://drive.google.com/uc?id=1xzNrYAqii3p-Y8XIiBd1ciSM4e1VRXIc
To: /content/Maquinaria_Pesada_Video.mp4
26.1MB [00:00, 62.8MB/s]


# Cargar Modelos Entrenados

## SSD

In [4]:
# Cargar utils files
!gdown https://drive.google.com/uc?id=1vIN5yU7c1aQ-47JRHmGmFafjNk5-SkoP
!unzip ssd-utils-master.zip
!mv  -v /content/ssd-utils-master/* /content/

Downloading...
From: https://drive.google.com/uc?id=1vIN5yU7c1aQ-47JRHmGmFafjNk5-SkoP
To: /content/ssd-utils-master.zip
100% 12.8k/12.8k [00:00<00:00, 25.1MB/s]
Archive:  ssd-utils-master.zip
  inflating: ssd-utils-master/detect.py  
  inflating: ssd-utils-master/model.py  
  inflating: ssd-utils-master/utils.py  
renamed '/content/ssd-utils-master/detect.py' -> '/content/detect.py'
renamed '/content/ssd-utils-master/model.py' -> '/content/model.py'
renamed '/content/ssd-utils-master/utils.py' -> '/content/utils.py'


In [5]:
# Download model
!gdown https://drive.google.com/uc?id=1x-ub_KArdhCQEFC5lQIRLjbQIN_W-Zio
!gdown https://drive.google.com/uc?id=1xo4CKksBUlfNXsYYptl75YPCmvYIrlDH
!gdown https://drive.google.com/uc?id=1HBDFQy8hROa4Iv31cNXZu69hP4QiSSms

Downloading...
From: https://drive.google.com/uc?id=1x-ub_KArdhCQEFC5lQIRLjbQIN_W-Zio
To: /content/modelssdCOPY.pth
99.1MB [00:01, 88.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xo4CKksBUlfNXsYYptl75YPCmvYIrlDH
To: /content/label2targetCOPY.txt
100% 161/161 [00:00<00:00, 290kB/s]
Downloading...
From: https://drive.google.com/uc?id=1HBDFQy8hROa4Iv31cNXZu69hP4QiSSms
To: /content/target2labelCOPY.txt
100% 161/161 [00:00<00:00, 274kB/s]


In [15]:
## Load Model SSD
from model import SSD300, MultiBoxLoss
from detect import *
import pickle
target2label_ssd = {}

with open('target2labelCOPY.txt','rb') as fp:
  target2label_ssd = pickle.load(fp)

model = torch.load(
    'modelssdCOPY.pth',
    map_location=torch.device(device))

def detect_image_ssd(original_image):
      bbs, labels, scores = detect(original_image, model, min_score=0.9, max_overlap=0.5,top_k=200, device=device)
      #labels = [target2label_ssd[c.item()] for c in labels]
      detections = [ [*bb, score,cls] for bb,score,cls in zip(bbs,scores,labels)]
      detections = torch.tensor(detections)
      return detections  #x1, y1, x2, y2, conf, cls_conf, cls_pred

In [18]:
import time
import datetime
vs = cv2.VideoCapture('Maquinaria_Pesada_Video.mp4')
nms_thres=0.1
conf_thres = 0.1
writer = None
i = 1
img_size = 300
classes = target2label_ssd
unique_labels = np.array(list(classes.keys()))
# generar colores para los bounding-box
cmap = plt.get_cmap('gist_ncar')
colors = [cmap(i) for i in np.linspace(0, 1, 20)]
colors = np.array(colors)
while i < 2500:
  (grab, frame) = vs.read()
  i +=1
  if not grab:
    break
  prev_time = time.time()
  newImg = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
  img = Image.fromarray(newImg)
  detections = detect_image_ssd(img)
  inference_time = datetime.timedelta(seconds=time.time() - prev_time)
  print ('Inference Time: %s' % (inference_time))

  img = np.array(img)
  
  if detections is not None:

    for x1, y1, x2, y2, cls_conf, cls_pred in detections:
      color = colors[int(np.where(unique_labels == int(cls_pred))[0])]
      color = np.array(color) * 255
      color = color.tolist()
      newImg = cv2.rectangle(newImg,(x1,y1),(x2,y2),color,2)
      newImg = cv2.putText(newImg,classes[int(cls_pred)],(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.7,color)
    if writer is None:
      fourcc = cv2.VideoWriter_fourcc(*'MP4V')
      writer = cv2.VideoWriter('ssd_output.mp4',fourcc,30,(newImg.shape[1],newImg.shape[0]))
    writer.write(cv2.cvtColor(newImg,cv2.COLOR_RGB2BGR))
writer.release() 

Inference Time: 0:00:00.094730
Inference Time: 0:00:00.093359
Inference Time: 0:00:00.093103
Inference Time: 0:00:00.085261
Inference Time: 0:00:00.084443
Inference Time: 0:00:00.088488
Inference Time: 0:00:00.081121
Inference Time: 0:00:00.076781
Inference Time: 0:00:00.069803
Inference Time: 0:00:00.070232
Inference Time: 0:00:00.071081
Inference Time: 0:00:00.068300
Inference Time: 0:00:00.066899
Inference Time: 0:00:00.066884
Inference Time: 0:00:00.067832
Inference Time: 0:00:00.072462
Inference Time: 0:00:00.069760
Inference Time: 0:00:00.071291
Inference Time: 0:00:00.068212
Inference Time: 0:00:00.065528
Inference Time: 0:00:00.072797
Inference Time: 0:00:00.067595
Inference Time: 0:00:00.066965
Inference Time: 0:00:00.067658
Inference Time: 0:00:00.069587
Inference Time: 0:00:00.069660
Inference Time: 0:00:00.067585
Inference Time: 0:00:00.068778
Inference Time: 0:00:00.071194
Inference Time: 0:00:00.071592
Inference Time: 0:00:00.070168
Inference Time: 0:00:00.069921
Inferenc

## FasterRCNN

In [9]:
# Download model
!gdown https://drive.google.com/uc?id=1yYYr31tDb_tU6Xhrg6msHUdxOh6EZvvY
!gdown https://drive.google.com/uc?id=1v_n-m0mh7VNrFf2sBDby4tAiUJSUYQwt
!gdown https://drive.google.com/uc?id=16_i3sFQpmHX2kD0P7fPmD55P4Y70NHaz

Downloading...
From: https://drive.google.com/uc?id=1yYYr31tDb_tU6Xhrg6msHUdxOh6EZvvY
To: /content/model_fastetr_rcnn_50epochs_SGD.pth
166MB [00:02, 80.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1v_n-m0mh7VNrFf2sBDby4tAiUJSUYQwt
To: /content/target2label_SGD.txt
100% 133/133 [00:00<00:00, 219kB/s]
Downloading...
From: https://drive.google.com/uc?id=16_i3sFQpmHX2kD0P7fPmD55P4Y70NHaz
To: /content/label2target_SGD.txt
100% 133/133 [00:00<00:00, 222kB/s]


In [20]:
# Load Model
import pickle
target2label_fasterrcnn = {}

with open('target2label_SGD.txt','rb') as fp:
  target2label_fasterrcnn = pickle.load(fp)

with open('label2target_SGD.txt','rb') as fp:
  label2target_fasterrcnn = pickle.load(fp)

model = torch.load(
    'model_fastetr_rcnn_50epochs_SGD.pth',
    map_location=torch.device(device))


def detect_image_fasterrcnn(image):
  outputs = model([image])
  pred_bbs, pred_confs, pred_labels = decode_output(outputs[0])
  detections =  [[*pred_bb,conf,label2target_fasterrcnn[pred_cls]] 
                  for pred_cls, conf, pred_bb in zip( pred_labels ,pred_confs, pred_bbs)]
  detections = torch.tensor(detections)
  return detections  #x1, y1, x2, y2, conf, cls_conf, cls_pred

In [21]:
from torchvision.ops import nms
def decode_output(output):
    'convert tensors to numpy arrays'
    bbs = output['boxes'].cpu().detach().numpy().astype(np.uint16)
    labels = np.array([target2label_fasterrcnn[i] for i in output['labels'].cpu().detach().numpy()])
    confs = output['scores'].cpu().detach().numpy()
    ixs = nms(torch.tensor(bbs.astype(np.float32)), torch.tensor(confs), 0.05)
    bbs, confs, labels = [tensor[ixs] for tensor in [bbs, confs, labels]]

    if len(ixs) == 1:
        bbs, confs, labels = [np.array([tensor]) for tensor in [bbs, confs, labels]]
    return bbs.tolist(), confs.tolist(), labels.tolist()

In [22]:
def preprocess_image(img):
    img = np.array(img.resize((400, 400), resample=Image.BILINEAR))/255.
    img = torch.tensor(img).permute(2,0,1)
    return img.to(device).float()

In [23]:
import time
import datetime
vs = cv2.VideoCapture('Maquinaria_Pesada_Video.mp4')
nms_thres=0.1
conf_thres = 0.1
writer = None
i = 1
img_size = 400
classes = target2label_fasterrcnn
unique_labels = np.array(list(classes.keys()))
# generar colores para los bounding-box
cmap = plt.get_cmap('gist_ncar')
colors = [cmap(i) for i in np.linspace(0, 1, 20)]
colors = np.array(colors)
while i < 2500:
  (grab, frame) = vs.read()
  i +=1
  if not grab:
    break
  prev_time = time.time()
  newImg = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
  temp_img = Image.fromarray(newImg)
  prep_img = preprocess_image(temp_img)
  detections = detect_image_fasterrcnn(prep_img)
  #print(detections)
  img = np.array(newImg)
  H,W, _ = img.shape 
  inference_time = datetime.timedelta(seconds=time.time() - prev_time)
  print ('Inference Time: %s' % (inference_time))

  if detections is not None:
    for x1, y1, x2, y2, cls_conf, cls_pred in detections:
      x1 = int(x1 * W/ img_size)
      x2 = int(x2 * W/ img_size)
      y1 = int(y1 * H/ img_size)
      y2 = int(y2 * H/ img_size)
      color = colors[int(np.where(unique_labels == int(cls_pred))[0])]
      color = np.array(color) * 255
      color = color.tolist()
      newImg = cv2.rectangle(newImg,(x1,y1),(x2,y2),color,2)
      newImg = cv2.putText(newImg,classes[int(cls_pred)],(x1,y1),cv2.FONT_HERSHEY_COMPLEX,0.7,color)
    if writer is None:
      fourcc = cv2.VideoWriter_fourcc(*'MP4V')
      writer = cv2.VideoWriter('fasterrcnn_output.mp4',fourcc,30,(newImg.shape[1],newImg.shape[0]))
    writer.write(cv2.cvtColor(newImg,cv2.COLOR_RGB2BGR))
writer.release() 

Inference Time: 0:00:00.350739
Inference Time: 0:00:00.305956
Inference Time: 0:00:00.272943
Inference Time: 0:00:00.280765
Inference Time: 0:00:00.286184
Inference Time: 0:00:00.272309
Inference Time: 0:00:00.280055
Inference Time: 0:00:00.275441
Inference Time: 0:00:00.277646
Inference Time: 0:00:00.284525
Inference Time: 0:00:00.273996
Inference Time: 0:00:00.283719
Inference Time: 0:00:00.277111
Inference Time: 0:00:00.282114
Inference Time: 0:00:00.284827
Inference Time: 0:00:00.278623
Inference Time: 0:00:00.278122
Inference Time: 0:00:00.278071
Inference Time: 0:00:00.286334
Inference Time: 0:00:00.276108
Inference Time: 0:00:00.281565
Inference Time: 0:00:00.283805
Inference Time: 0:00:00.273403
Inference Time: 0:00:00.272984
Inference Time: 0:00:00.274881
Inference Time: 0:00:00.276995
Inference Time: 0:00:00.282336
Inference Time: 0:00:00.275038
Inference Time: 0:00:00.280733
Inference Time: 0:00:00.278606
Inference Time: 0:00:00.276842
Inference Time: 0:00:00.278196
Inferenc